* col, expr, cast, alias
* lit
* concat, concat_ws
* upper, lower, initcap
* length
* split, explode
* lpad, rpad
* ltrim, rtrim, trim

* type, size (array length)

* current_date, current_timestamp, to_date, to_timestamp
* date_add, date_sub, datediff, months_between, add_months, next_day
* trunc, date_trunc
* year, month, weekeofyear, dayofyear, dayofmonth, dayofweek, hour minute second
* date_format
* unix_timestamp, from_unixtime

* coalesce, nvl 
* sql, selectExpr, expr

* na, fillna, replacena, dropna, isnan, isin
* isNull, isNotNull
* distinct, drop_duplicates, dropDuplicates
* df.na.drop, df.dropna

* sort, orderBy
* desc, asc, col -> asc(), asc_nulls_first(), asc_nulls_last(), desc(), desc_nulls_first(), desc_nulls_last()
* agg

## Actions - immediate execution
count, show, 

postponed execution until the action
select, 

## Parameters

`spark.conf.get()`, `spark.conf.set()`

* `spark.sql.repl.eagerEval.enabled` 
* `spark.sql.autoBroadcastJoinThreshold` - default 
* `spark.sql.shuffle.partitions` - default 200

